# Predictor del Valor de Mercado de Autos Usados

-----

## Overview

### Descripción

<div style="color: #196CC4;">
    El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado. A Rusty Bargain le interesa:<br>

▶ la calidad de la predicción;<br>
▶ la velocidad de la predicción;<br>
▶ el tiempo requerido para el entrenamiento
 </div>

### Objetivo

<div style="color: #196CC4;">Desarrollar un modelo de predicción de precios de vehículos de segunda mano para Rusty Bargain, con el fin de proporcionar a los usuarios una estimación precisa y rápida del valor de mercado de sus coches. El modelo debe cumplir con los criterios de calidad establecidos por Rusty Bargain, garantizando una predicción precisa y una velocidad de ejecución adecuada para mejorar la experiencia del usuario en la plataforma. </div>

### Recursos

<div style="color: #196CC4;">
<b>Características</b><br>
▶ DateCrawled — fecha en la que se descargó el perfil de la base de datos<br>
▶ VehicleType — tipo de carrocería del vehículo<br>
▶ RegistrationYear — año de matriculación del vehículo<br>
▶ Gearbox — tipo de caja de cambios<br>
▶ Power — potencia (CV)<br>
▶ Model — modelo del vehículo<br>
▶ Mileage — kilometraje (medido en km de acuerdo con las especificidades regionales del conjunto de datos)<br>
▶ RegistrationMonth — mes de matriculación del vehículo<br>
▶ FuelType — tipo de combustible<br>
▶ Brand — marca del vehículo<br>
▶ NotRepaired — vehículo con o sin reparación<br>
▶ DateCreated — fecha de creación del perfil<br>
▶ NumberOfPictures — número de fotos del vehículo<br>
▶ PostalCode — código postal del propietario del perfil (usuario)<br>
▶ LastSeen — fecha de la última vez que el usuario estuvo activo<br>

<b>Objetivo</b><br>
▶ Price — precio (en euros)<br>
</div>

### Metodología

<div style="color: #196CC4;">
<ol>
<li><strong>Inicialización y Análisis Exploratorio de Datos</strong>
<ul>
<li>Se importan las librerías, módulos y el conjunto de datos: car_data.csv.</li>
<li>Se realiza un análisis exploratorio de los datos, incluyendo correlaciones preliminares.</li>
<li>Se calculan estadísticas descriptivas.</li>
<li>Se identifican y tratan valores duplicados y faltantes.</li>
<li>Se eliminan las variables que no aportan valor al modelo.</li>
<li>Se normalizan las características para asegurar una escala común.</li>
<li>Se realiza codificación de variables categóricas (One-Hot Encoding y Label Encoding).</li>
<li>Se divide el conjunto de datos en conjuntos de entrenamiento, validación y prueba (proporción 3:1:1).</li>
</ul>
</li>

<li><strong>Entrenamiento de Modelos</strong>
<ul>
<li>Se entrenan modelos con y sin potenciación de gradiente.</li>
<li>Modelos sin potenciación de gradiente: Árbol de Decisión, Regresión Lineal y Random Forest.</li>
<li>Modelos con potenciación de gradiente: LightGBM, CatBoost y XGBoost.</li>
<li>Se evalúan los modelos utilizando las métricas MSE, RMSE y tiempo de entrenamiento y predicción.</li>
</ul>
</li>

<li><strong>Evaluación y Comparación</strong>
<ul>
<li>Se presenta un resumen de las métricas obtenidas para cada modelo.</li>
<li>Se comparan los modelos en términos de calidad de predicción (MSE, RMSE, MAPE) y eficiencia (tiempo de entrenamiento y predicción).</li>
</ul>
</li>

<li><strong>Conclusiones</strong>
<ul>
<li>Se selecciona el modelo óptimo basado en el rendimiento y los requerimientos del proyecto.</li>
</ul>
</li>
</ol>

</div>

-----

## Información general

### Inicialización

In [38]:
# Data analysis
import pandas as pd

# Label Encoding for categories
from sklearn.preprocessing import LabelEncoder

# Testing sets
from sklearn.model_selection import train_test_split

# Model training
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Gradient boosting model training
import lightgbm as lgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import time

from sklearn.preprocessing import MinMaxScaler

In [39]:
# Import data
cars = pd.read_csv('datasets/car_data.csv')

### Despliegue de dataset

<div style="color: #196CC4;">
▶ Propiedades generales del Dataframe
</div>

In [40]:
# General Dataframe properties
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

<div style="color: #196CC4;">
▶ Vistazo general del Dataframe
</div>

In [41]:
# General data overview
display(cars.head(3))

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47


<div style="color: #196CC4;">
▶ Verificación de valores duplicados.
</div>

In [42]:
# Duplicated values
cars_duplicates = cars.duplicated()

# Sum of duplicated values
total_cars_duplicates = cars_duplicates.sum()

# Duplicated rows
cars_duplicates_rows = cars[cars_duplicates]

# Display data
print("Total de valores duplicados:")
print(total_cars_duplicates)
print()
#print("Listado de aquellas filas duplicadas:")
#display(cars_duplicates_rows)

Total de valores duplicados:
262



<div style="color: #196CC4;">
▶ Estadísticas descriptivas para los datos numéricos.
</div>

In [43]:
# Descriptive statistics
cars.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


<div style="color: #196CC4;">
▶ En cuanto a las variables categóricas, a continuación se verifica la cantidad de valores para cada una:
</div>

In [44]:
# Unique values in 'DateCrawled'
unique_surname = cars['DateCrawled'].nunique()
print("Número de valores únicos en la serie 'DateCrawled':", unique_surname)

# Unique values in 'VehicleType'
unique_surname = cars['VehicleType'].nunique()
print("Número de valores únicos en la serie 'VehicleType':", unique_surname)

# Unique values in 'Gearbox'
unique_surname = cars['Gearbox'].nunique()
print("Número de valores únicos en la serie 'Gearbox':", unique_surname)

# Unique values in 'Model'
unique_surname = cars['Model'].nunique()
print("Número de valores únicos en la serie 'Model':", unique_surname)

# Unique values in 'FuelType'
unique_surname = cars['FuelType'].nunique()
print("Número de valores únicos en la serie 'FuelType':", unique_surname)

# Unique values in 'Brand'
unique_surname = cars['Brand'].nunique()
print("Número de valores únicos en la serie 'Brand':", unique_surname)

# Unique values in 'NotRepaired'
unique_surname = cars['NotRepaired'].nunique()
print("Número de valores únicos en la serie 'NotRepaired':", unique_surname)

# Unique values in 'DateCreated'
unique_surname = cars['DateCreated'].nunique()
print("Número de valores únicos en la serie 'DateCreated':", unique_surname)

# Unique values in 'LastSeen'
unique_surname = cars['LastSeen'].nunique()
print("Número de valores únicos en la serie 'LastSeen':", unique_surname)

Número de valores únicos en la serie 'DateCrawled': 15470
Número de valores únicos en la serie 'VehicleType': 8
Número de valores únicos en la serie 'Gearbox': 2
Número de valores únicos en la serie 'Model': 250
Número de valores únicos en la serie 'FuelType': 7
Número de valores únicos en la serie 'Brand': 40
Número de valores únicos en la serie 'NotRepaired': 2
Número de valores únicos en la serie 'DateCreated': 109
Número de valores únicos en la serie 'LastSeen': 18592


### Observaciones iniciales

<div style="color: #196CC4;">
<b>Observaciones iniciales:</b><br>
▶ Los nombres de las Series están en mayúsculas<br>
▶ Hay 262 filas duplicadas<br>
▶ Hay algunas series que no influyen en el propósito de este proyecto como: "DateCrawled", "Model" (ya que contamos con la marca del vehículo), "DateCreated" y "LastSeen"<br>
▶ Existen valores ausentes en "vehicletype", "gearbox", "fueltype" y "notrepaired"<br>
▶ Hay Series que son de tipo "object", por lo que se debe valorar el adecuado tratamiento de estos datos<br>
▶ De acuerdo a la estadística descriptiva, se muestran valores atípicos en las Series "Price", "RegistrationYear" y "Power"
</div>

-----

## Análisis Exploratorio de Datos (EDA)

### Limpieza de datos

<div style="color: #196CC4;">
▶ Se sugiere que los nombres de las columnas sean minusculas para tener mayor consistencia y sean más fáciles de trabajar
</div>

In [45]:
# Convert to lowercase
mapeo = {columna: columna.lower() for columna in cars.columns}

# Renaming DataFrame
cars = cars.rename(columns=mapeo)

<div style="color: #196CC4;">
<b>Valores duplicados</b><br>
▶ Los 262 datos duplicados que existen en "cars", son solo una pequeña parte del conjunto de datos y no afectan significativamente la representación general de los datos restante (354,369), por lo que se sugiere eliminar estos registros que podrían afectar el resultado del entrenamiento<br>
    
</div>

In [46]:
# Delete duplicates
cars = cars.drop_duplicates().reset_index(drop=True)

<div style="color: #196CC4;">
<b>Eliminación de Series sin afectar los resultados</b><br>
▶ <b>datecrawled:</b> Con 15,470 valores únicos; las fechas en las que se descargaron los perfiles de la base de datos  no serán tan importantes para el precio de los vehículos.<br>
▶ <b>model:</b> Con 250 valores únicos, el modelo puede ser demasiado detallado para ser útil en la predicción de precios, además ya tenemos la marca del vehículo.<br>
▶ <b>datecreated:</b> Con 109 valores únicos, la fecha de creación del perfil puede no ser relevante para predecir el precio de los vehículos<br>
▶ <b>lastseen:</b> Con 18,592 valores únicos, la fecha de la última vez que el usuario estuvo activo podría no tener una influencia directa en el precio de los vehículos.<br>
▶ <b>numberofpictures:</b> La cantidad de fotos no influye en los objetivo del proyecto.<br>
▶ <b>postalcode:</b> Igualmente el Código Postal no influye en los objetivo del proyecto.<br>    
</div>

In [47]:
# Useless Series
delete_series = ['datecrawled', 'model', 'datecreated', 'lastseen', 'numberofpictures', 'postalcode']

# Delete Series
cars = cars.drop(delete_series, axis=1)

# Verificar el DataFrame actualizado
print(cars.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354107 entries, 0 to 354106
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              354107 non-null  int64 
 1   vehicletype        316623 non-null  object
 2   registrationyear   354107 non-null  int64 
 3   gearbox            334277 non-null  object
 4   power              354107 non-null  int64 
 5   mileage            354107 non-null  int64 
 6   registrationmonth  354107 non-null  int64 
 7   fueltype           321218 non-null  object
 8   brand              354107 non-null  object
 9   notrepaired        282962 non-null  object
dtypes: int64(5), object(5)
memory usage: 27.0+ MB
None


<div style="color: #196CC4;">
<b>Tratamiento de valores ausentes</b><br>
▶ <b>Eliminación de filas:</b> El conjunto de datos tiene 354107 entradas y la cantidad de valores ausentes es notablemente menor en comparación, por lo que se están eliminando todas las filas que correspondan con valores duplicados para la Serie "vehicletype". Con este cambio quedarán 316623 entradas con las que se podrá seguir trabajando, correspondiente al 89.4% de la muestra original de datos.<br>
▶ <b>Imputacion:</b> El número de valores ausentes que restan es relativamente pequeño en comparación con el tamaño inicial cuando se tenía el conjunto total, por lo que para poder seguir trabajando con ese 89.4% de datos no reducir más la muestra de información valiosa, se imputarán los valores ausentes utilizando la moda (valor más frecuente) de cada serie para "gearbox", "fueltype" y "notrepaired". <br><br>
Eliminar parcialmente las filas con valores ausentes, ayuda a garantizar que datos restantes sean completos y estén listos para su análisis. De esta manera la cantidad de datos a imputar también se disminuye y con ello se reduce también el sesgo en los datos.
</div>

In [48]:
# Delete rows
cars.dropna(subset=['vehicletype'], inplace=True)

In [49]:
# Mode Imputation
cars['gearbox'].fillna(cars['gearbox'].mode()[0], inplace=True)
cars['fueltype'].fillna(cars['fueltype'].mode()[0], inplace=True)
cars['notrepaired'].fillna(cars['notrepaired'].mode()[0], inplace=True)

# Verificar el DataFrame actualizado
print(cars.info())

<class 'pandas.core.frame.DataFrame'>
Index: 316623 entries, 1 to 354106
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              316623 non-null  int64 
 1   vehicletype        316623 non-null  object
 2   registrationyear   316623 non-null  int64 
 3   gearbox            316623 non-null  object
 4   power              316623 non-null  int64 
 5   mileage            316623 non-null  int64 
 6   registrationmonth  316623 non-null  int64 
 7   fueltype           316623 non-null  object
 8   brand              316623 non-null  object
 9   notrepaired        316623 non-null  object
dtypes: int64(5), object(5)
memory usage: 26.6+ MB
None


C:\Users\dguez\AppData\Local\Temp\ipykernel_18400\1516463945.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cars['gearbox'].fillna(cars['gearbox'].mode()[0], inplace=True)
C:\Users\dguez\AppData\Local\Temp\ipykernel_18400\1516463945.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

<div style="color: #196CC4;">
<b>Outliers</b><br>
▶ A continuación se calculan los valores específicos para la media y la desviación estándar de cada variable, estos valores se pueden usar para ajustar los límites para identificar y eliminar los valores atípicos en las Series "price", "registrationyear" y "power".
</div>

In [50]:
# Mean and the Standard Deviation
mean_price = cars['price'].mean()
std_price = cars['price'].std()

mean_registrationyear = cars['registrationyear'].mean()
std_registrationyear = cars['registrationyear'].std()

mean_power = cars['power'].mean()
std_power = cars['power'].std()

# Print
print("Media de la columna 'price':", mean_price)
print("Desviación estandar de la columna 'price':", std_price)
print()
print("Media de la columna 'registrationyear':", mean_registrationyear)
print("Desviación estandar de la columna 'registrationyear':", std_registrationyear)
print()
print("Media de la columna 'power':", mean_power)
print("Desviación estandar de la columna 'power':", std_power)

Media de la columna 'price': 4658.036368172874
Desviación estandar de la columna 'price': 4584.561111519157

Media de la columna 'registrationyear': 2002.238640275659
Desviación estandar de la columna 'registrationyear': 6.566531306470144

Media de la columna 'power': 114.76780903471952
Desviación estandar de la columna 'power': 185.93147452990652


<div style="color: #196CC4;">
▶ Basándonos en estos valores, los límites para identificar los valores atípicos serán ±2 desviaciones estándar de la media.
</div>


In [51]:
# Limit Outliers
lower_bound_price = 4431.15 - 2 * 4261.85
upper_bound_price = 4431.15 + 2 * 4261.85

lower_bound_registrationyear = 2002.51 - 2 * 5.52
upper_bound_registrationyear = 2002.51 + 2 * 5.52

lower_bound_power = 110.71 - 2 * 60.40
upper_bound_power = 110.71 + 2 * 60.40

<div style="color: #196CC4;">
▶ A continuación se utilizan estos límites para identificar y eliminar los valores atípicos, como se muestra a continuación:
</div>

In [52]:
# Delete Outliers
cars = cars[(cars['price'] >= lower_bound_price) & (cars['price'] <= upper_bound_price)]
cars = cars[(cars['registrationyear'] >= lower_bound_registrationyear) & (cars['registrationyear'] <= upper_bound_registrationyear)]
cars = cars[(cars['power'] >= lower_bound_power) & (cars['power'] <= upper_bound_power)]

In [53]:
# Descriptive statistics
cars.describe()

,price,registrationyear,power,mileage,registrationmonth
count,271236.000000,271236.000000,271236.000000,271236.000000,271236.000000
mean,3587.375724,2002.390604,103.115774,132129.252754,5.935801
std,3139.455275,4.713636,51.284131,33332.441315,3.636323
min,0.000000,1992.000000,0.000000,5000.000000,0.000000
25%,1100.000000,1999.000000,71.000000,125000.000000,3.000000
50%,2500.000000,2002.000000,102.000000,150000.000000,6.000000
75%,5350.000000,2006.000000,140.000000,150000.000000,9.000000
max,12950.000000,2013.000000,231.000000,150000.000000,12.000000


<div style="color: #196CC4;">
<b>Escalación de datos</b><br>
▶ La técnica de escalar datos es comúnmente utilizada en el preprocesamiento de datos para poner todas las características (features) en una misma escala. <br>
▶ A continuación se escalan las características 'price', 'power' y 'mileage' en el conjunto de datos, para normalizar características.
</div>

In [54]:
# Inicializer
escalador = MinMaxScaler()

# Transformation
#cars[['price','power', 'mileage']] = escalador.fit_transform(cars[['price', 'power', 'mileage']])
cars[['power', 'mileage']] = escalador.fit_transform(cars[['power', 'mileage']])

In [55]:
display(cars[['power', 'mileage']])
display(cars)

,power,mileage
2,0.705628,0.827586
3,0.324675,1.000000
4,0.298701,0.586207
5,0.441558,1.000000
6,0.471861,1.000000
...,...,...
354100,0.974026,1.000000
354101,0.000000,1.000000
354104,0.437229,0.827586
354105,0.441558,1.000000


,price,vehicletype,registrationyear,gearbox,power,mileage,registrationmonth,fueltype,brand,notrepaired
2,9800,suv,2004,auto,0.705628,0.827586,8,gasoline,jeep,no
3,1500,small,2001,manual,0.324675,1.000000,6,petrol,volkswagen,no
4,3600,small,2008,manual,0.298701,0.586207,7,gasoline,skoda,no
5,650,sedan,1995,manual,0.441558,1.000000,10,petrol,bmw,yes
6,2200,convertible,2004,manual,0.471861,1.000000,8,petrol,peugeot,no
...,...,...,...,...,...,...,...,...,...,...
354100,3200,sedan,2004,manual,0.974026,1.000000,5,petrol,seat,yes
354101,1150,bus,2000,manual,0.000000,1.000000,3,petrol,opel,no
354104,1199,convertible,2000,auto,0.437229,0.827586,3,petrol,smart,no
354105,9200,bus,1996,manual,0.441558,1.000000,3,gasoline,volkswagen,no


<div style="color: #196CC4;">
<b>Labeling para el tratamiento de las Series restantes con tipo de valor "object"</b><br>
▶ One-Hot Encoding (OHE): Con columnas binarias para cada categoría. Esto se aplica en las series 'vehicletype', 'gearbox', 'fueltype', 'notrepaired' ya que tienen entre 2 y 8 valores.<br>
▶ Codificación de etiquetas (Label Encoding): Asignando un número entero a cada categoría única. Esto se aplica en la serie 'brand'.
</div>

In [56]:
# OHE for Encoding
cars = pd.get_dummies(cars, columns=['vehicletype', 'gearbox', 'fueltype', 'notrepaired'])

In [57]:
# Label Encoding for 'Brand'
label_encoder = LabelEncoder()
cars['brand'] = label_encoder.fit_transform(cars['brand'])

### Despliegue de información

<div style="color: #196CC4;">
▶ A continuación verifico los cambios realizados en las propiedades del DataFrame 'cars' y su previsualización.<br>
</div>

In [58]:
# Data displayment
cars.info()
display(cars.head(5))

<class 'pandas.core.frame.DataFrame'>
Index: 271236 entries, 2 to 354106
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   price                    271236 non-null  int64  
 1   registrationyear         271236 non-null  int64  
 2   power                    271236 non-null  float64
 3   mileage                  271236 non-null  float64
 4   registrationmonth        271236 non-null  int64  
 5   brand                    271236 non-null  int32  
 6   vehicletype_bus          271236 non-null  bool   
 7   vehicletype_convertible  271236 non-null  bool   
 8   vehicletype_coupe        271236 non-null  bool   
 9   vehicletype_other        271236 non-null  bool   
 10  vehicletype_sedan        271236 non-null  bool   
 11  vehicletype_small        271236 non-null  bool   
 12  vehicletype_suv          271236 non-null  bool   
 13  vehicletype_wagon        271236 non-null  bool   
 14  gearbox_a

,price,registrationyear,power,mileage,registrationmonth,brand,vehicletype_bus,vehicletype_convertible,vehicletype_coupe,vehicletype_other,...,gearbox_manual,fueltype_cng,fueltype_electric,fueltype_gasoline,fueltype_hybrid,fueltype_lpg,fueltype_other,fueltype_petrol,notrepaired_no,notrepaired_yes
2,9800,2004,0.705628,0.827586,8,14,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
3,1500,2001,0.324675,1.000000,6,38,False,False,False,False,...,True,False,False,False,False,False,False,True,True,False
4,3600,2008,0.298701,0.586207,7,31,False,False,False,False,...,True,False,False,True,False,False,False,False,True,False
5,650,1995,0.441558,1.000000,10,2,False,False,False,False,...,True,False,False,False,False,False,False,True,False,True
6,2200,2004,0.471861,1.000000,8,25,False,True,False,False,...,True,False,False,False,False,False,False,True,True,False


### Inicialización para los modelos

<div style="color: #196CC4;">
<b>A continuación se dividen los datos fuente de la siguiente manera para llegar a una proporción de 3:1:1:</b><br>
▶ 60% Dataset para el entrenamiento<br>
▶ 20% Dataset de validación<br>
▶ 20% Dataset de prueba<br><br>
Cuanto más grande sea el conjunto de entrenamiento, más datos tendrá el modelo para aprender patrones y relaciones en los datos. Por otro lado el dataset de validación servirá para evaluar el rendimiento del modelo y el dataset de prueba esperará al final para evaluar el rendimiento final del modelo  (cuando haya terminado el proceso de entrenamiento y validación).
</div>

In [59]:
# Split the data into a training set (80%) and a test set (20%)
#df_train, df_test = train_test_split(cars, test_size=0.8, random_state=12345)

# Split the data into a training set (60%), a validation set (20%), and a test set (20%)
df_train, remaining_data = train_test_split(cars, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(remaining_data, test_size=0.5, random_state=12345)

<div style="color: #196CC4;">
▶ En estas líneas de código se están preparando los datos para el entrenamiento y la prueba de un modelo de aprendizaje automático. Se realiza una separación entre las características (features) y la variable objetivo (target) en cada conjunto de datos.<br>
</div>

In [60]:
# Extract features and target variables from the training and validation sets
features_train = df_train.drop(['price'], axis=1)
target_train = df_train['price']

features_valid = df_valid.drop(['price'], axis=1)
target_valid = df_valid['price']

# FINAL TESTING SETS ↓

# Train + Valid OHE
total_features_train = pd.concat([features_train, features_valid])
total_target_train = pd.concat([target_train, target_valid])

features_test = df_test.drop(['price'], axis=1)
target_test = df_test['price']

-----

## Entrenamiento sin Potenciación de Gradiente

<div style="color: #196CC4;">
A continuación se describe el proceso de entrenamiento y evaluación de tres modelos de aprendizaje automático sin potenciación de gradiente: <br>
▶ DecisionTree <br>
▶ Regresión Lineal<br>
▶ Random Forest<br>

Con estos datos históricos se podrán predecir los precios de automóviles y estos resultados se evalúan en función de tres criterios principales: <br>
▶ Calidad de la predicción a través de MSE y RMSE<br>
▶ Tiempo de entrenamiento<br>
▶ Tiempo de predicción.</div>

### Modelo DecisionTree

<div style="color: #196CC4;">
▶ Un árbol de decisión es un modelo que toma decisiones dividiendo los datos en segmentos basados en preguntas o reglas simples. Cada "nodo" en el árbol representa una pregunta sobre una característica, cada "rama" es el resultado de esa pregunta, y cada "hoja" es una predicción final.</div>

In [61]:
# Hyperparameters
max_depth = 5
min_samples_split = 2

# Train
tree_start_train_time = time.time()
decisiontree = DecisionTreeRegressor(random_state=12345, max_depth=max_depth, min_samples_split=min_samples_split)
decisiontree.fit(features_train, target_train)

# Training time
tree_end_train_time = time.time()
tree_train_time = tree_end_train_time - tree_start_train_time

# Price prediction
tree_start_pred_time = time.time()
decisiontree_pred = decisiontree.predict(features_valid)

# Predictions time
tree_end_pred_time = time.time()
tree_pred_time = tree_end_pred_time - tree_start_pred_time

In [62]:
# Quality of prediction metrics
tree_mse = mean_squared_error(target_valid, decisiontree_pred)
tree_rmse = tree_mse ** 0.5
tree_mape = mean_absolute_percentage_error(target_valid, decisiontree_pred)

# Print DecisionTree Model
print("Modelo DecisionTree")
print()

# Print metrics
print("Calidad de predicciones:")
print("Mean Squared Error (MSE):", tree_mse)
print("Root Mean Squared Error (RMSE):", tree_rmse)
print("Mean Absolute Percentage Error (MAPE):", tree_mape)
print()

# Print time
print("Tiempo de entrenamiento:", tree_train_time, "seconds")
print("Tiempo de predicción:", tree_pred_time, "seconds")

Modelo DecisionTree

Calidad de predicciones:
Mean Squared Error (MSE): 2975726.1010329495
Root Mean Squared Error (RMSE): 1725.0293043983193
Mean Absolute Percentage Error (MAPE): 2.325411609911504e+17

Tiempo de entrenamiento: 0.1955549716949463 seconds
Tiempo de predicción: 0.009880542755126953 seconds


### Regresión lineal

<div style="color: #196CC4;">
▶ La regresión lineal es un modelo que asume que hay una relación recta entre las características de un objeto y su valor, de esta forma trata de encontrar la línea recta que mejor se ajusta a los datos para hacer predicciones. Si pensamos en un gráfico, la regresión lineal busca la línea que mejor describa cómo cambia el precio a medida que se modifican los parámetros del automóvil.</div>

In [63]:
# Model
linear_reg = LinearRegression()

# Train
rlineal_start_train_time = time.time()
linear_reg.fit(features_train, target_train)

# Training time
rlineal_end_train_time = time.time()
rlineal_train_time = rlineal_end_train_time - rlineal_start_train_time

# Price prediction
rlineal_start_pred_time = time.time()
linear_reg_pred = linear_reg.predict(features_valid)

# Predictions time
rlineal_end_pred_time = time.time()
rlineal_pred_time = rlineal_end_pred_time - rlineal_start_pred_time

In [64]:
# Quality of prediction metrics
rlineal_mse = mean_squared_error(target_valid, linear_reg_pred)
rlineal_rmse = rlineal_mse ** 0.5
rlineal_mape = mean_absolute_percentage_error(target_valid, linear_reg_pred)

# Print LinealRegression
print("Modelo Regresión lineal")
print()

# Print metrics
print("Calidad de predicciones:")
print("Mean Squared Error (MSE):", rlineal_mse)
print("Root Mean Squared Error (RMSE):", rlineal_rmse)
print("Mean Absolute Percentage Error (MAPE):", rlineal_mape)
print()

# Print time
print("Tiempo de entrenamiento:", rlineal_train_time, "seconds")
print("Tiempo de predicción:", rlineal_pred_time, "seconds")

Modelo Regresión lineal

Calidad de predicciones:
Mean Squared Error (MSE): 3425450.8321811836
Root Mean Squared Error (RMSE): 1850.797350382041
Mean Absolute Percentage Error (MAPE): 2.388739192612065e+17

Tiempo de entrenamiento: 0.2594165802001953 seconds
Tiempo de predicción: 0.011917591094970703 seconds


### Random Forest

<div style="color: #196CC4;">
▶ El Random Forest es un conjunto de muchos árboles de decisión que trabajan juntos. Cada árbol se entrena con diferentes partes del conjunto de datos y sus resultados se combinan para hacer una predicción más precisa y robusta.</div>

In [65]:
# Parameters
param_dist_forest = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [1, 3, 5],
    'min_samples_split': [2, 4, 6]
}

# Randomized search & cross-validation
random_search_forest = RandomizedSearchCV(RandomForestRegressor(random_state=12345), param_dist_forest, scoring='neg_mean_squared_error', cv=2, n_iter=10)

# Training
forest_start_train_time = time.time()
random_search_forest.fit(features_train, target_train)

# Training time
forest_end_train_time = time.time()
forest_train_time = forest_end_train_time - forest_start_train_time

# Best model
best_model_forest = random_search_forest.best_estimator_

# Predictions
forest_start_pred_time = time.time()
predicted_valid_forest_random = best_model_forest.predict(features_valid)

# Predictions time
forest_end_pred_time = time.time()
forest_pred_time = forest_end_pred_time - forest_start_pred_time

In [66]:
# Quality of prediction metrics
forest_mse = mean_squared_error(target_valid, predicted_valid_forest_random)
forest_rmse = forest_mse ** 0.5
forest_mape = mean_absolute_percentage_error(target_valid, predicted_valid_forest_random)

# Print Random Forest
print("Modelo Random Forest")
print()

# Print metrics
print("Calidad de predicciones:")
print("Mean Squared Error (MSE):", forest_mse)
print("Root Mean Squared Error (RMSE):", forest_rmse)
print("Mean Absolute Percentage Error (MAPE):", forest_mape)
print()

# Print time
print("Tiempo de entrenamiento:", forest_train_time, "seconds")
print("Tiempo de predicción:", forest_pred_time, "seconds")

Modelo Random Forest

Calidad de predicciones:
Mean Squared Error (MSE): 2414588.9269462116
Root Mean Squared Error (RMSE): 1553.8947605762146
Mean Absolute Percentage Error (MAPE): 2.260626719437647e+17

Tiempo de entrenamiento: 134.9140751361847 seconds
Tiempo de predicción: 0.3460354804992676 seconds


-----

## Entrenamiento con Potenciación de Gradiente

<div style="color: #196CC4;">
<b>Potenciación del gradiente</b> es una técnica utilizada en aprendizaje automático para mejorar la precisión de los modelos de predicción, como los árboles de decisión. En lugar de entrenar un solo árbol y esperar que sea perfecto, la potenciación del gradiente trabaja en conjunto con muchos árboles pequeños, cada uno de los cuales aprende de los errores del anterior.<br>

A continuación se describe el proceso de entrenamiento y evaluación de tres modelos de aprendizaje automático con potenciación de gradiente: <br>
▶ Light Gradient Boosting Machine (LGBM)<br>
▶ CatBoost<br>
▶ XGBoost<br>

De igual forma con estos datos históricos se podrán predecir los precios de automóviles y estos resultados se evalúan en función de tres criterios principales: <br>
▶ Calidad de la predicción a través de MSE y RMSE<br>
▶ Tiempo de entrenamiento<br>
▶ Tiempo de predicción.</div>
</div>

### LGBM

<div style="color: #196CC4;">
▶ <b>LGBM</b> es un algoritmo de potenciación de gradiente que también utiliza árboles de decisión; es muy eficiente y rápido, ideal para grandes conjuntos de datos. LGBM también entrena árboles uno tras otro, cada uno aprendiendo de los errores del anterior para mejorar la predicción.</div>

In [67]:
# Hyperparameters
lgbm = LGBMRegressor(
    objective='regression',
    num_leaves=35,
    seed=23
)

# Training
lgbm_start_train_time = time.time()
lgbm.fit(features_train, target_train)

# Training time
lgbm_end_train_time = time.time()
lgbm_train_time = lgbm_end_train_time - lgbm_start_train_time

# Predictions
lgbm_start_pred_time = time.time()
lgbm_predicted_valid = lgbm.predict(features_valid)

# Predictions time
lgbm_end_pred_time = time.time()
lgbm_pred_time = lgbm_end_pred_time - lgbm_start_pred_time

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003102 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 346
[LightGBM] [Info] Number of data points in the train set: 162741, number of used features: 24
[LightGBM] [Info] Start training from score 3588.844753


In [68]:
# Quality of prediction metrics
lgbm_mse = mean_squared_error(target_valid, lgbm_predicted_valid)
lgbm_rmse = lgbm_mse ** 0.5
lgbm_mape = mean_absolute_percentage_error(target_valid, lgbm_predicted_valid)

# Print LGBM
print("Modelo LGBM")
print()

# Print metrics
print("Calidad de predicciones:")
print("Mean Squared Error (MSE):", lgbm_mse)
print("Root Mean Squared Error (RMSE):", lgbm_rmse)
print("Mean Absolute Percentage Error (MAPE):", lgbm_mape)
print()

# Print time
print("Tiempo de entrenamiento:", lgbm_train_time, "seconds")
print("Tiempo de predicción:", lgbm_pred_time, "seconds")

Modelo LGBM

Calidad de predicciones:
Mean Squared Error (MSE): 1651852.9482498362
Root Mean Squared Error (RMSE): 1285.2443146148657
Mean Absolute Percentage Error (MAPE): 2.1233470751532112e+17

Tiempo de entrenamiento: 0.44679689407348633 seconds
Tiempo de predicción: 0.04713726043701172 seconds


### CatBoost

<div style="color: #196CC4;">
▶ <b>CatBoost</b> es un algoritmo de aprendizaje automático diseñado específicamente para problemas de regresión y clasificación. Utiliza un método de aumento de gradiente que entrena una serie de árboles de decisión. 
</div>

In [69]:
# Hyperparameters
catboost = CatBoostRegressor(
    loss_function='RMSE',
    iterations=100,
    learning_rate=0.05,
    depth=6,
    random_seed=23
)

# Training
catboost_start_train_time = time.time()
catboost.fit(features_train, target_train, verbose=False)

# Training time
catboost_end_train_time = time.time()
catboost_train_time = catboost_end_train_time - catboost_start_train_time

# Predictions
catboost_start_pred_time = time.time()
catboost_predicted_valid = catboost.predict(features_valid)

# Predictions time
catboost_end_pred_time = time.time()
catboost_pred_time = catboost_end_pred_time - catboost_start_pred_time


In [70]:
# Quality of prediction metrics
catboost_mse = mean_squared_error(target_valid, catboost_predicted_valid)
catboost_rmse = catboost_mse ** 0.5
catboost_mape = mean_absolute_percentage_error(target_valid, catboost_predicted_valid)

# Print CatBoost
print("Modelo CatBoost")
print()

# Print metrics
print("Calidad de predicciones:")
print("Mean Squared Error (MSE):", catboost_mse)
print("Root Mean Squared Error (RMSE):", catboost_rmse)
print("Mean Absolute Percentage Error (MAPE):", catboost_mape)
print()

# Print time
print("Tiempo de entrenamiento:", catboost_train_time, "seconds")
print("Tiempo de predicción:", catboost_pred_time, "seconds")

Modelo CatBoost

Calidad de predicciones:
Mean Squared Error (MSE): 2001682.4135732444
Root Mean Squared Error (RMSE): 1414.8082603565913
Mean Absolute Percentage Error (MAPE): 2.2214954381836336e+17

Tiempo de entrenamiento: 0.7473475933074951 seconds
Tiempo de predicción: 0.0 seconds


### XGBoost

<div style="color: #196CC4;">
▶ <b>XGBoost</b> es una implementación optimizada de algoritmos de gradient boosting diseñada para mejorar la precisión y eficiencia de los modelos de aprendizaje automático. Su capacidad para manejar datos dispersos y su velocidad de entrenamiento lo hacen adecuado para conjuntos de datos complejos como el nuestro.
</div>

In [71]:
# Hyperparameters
xgboost = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.05,
    max_depth=6,
    random_state=23
)

# Training
xgboost_start_train_time = time.time()
xgboost.fit(features_train, target_train)

# Training time
xgboost_end_train_time = time.time()
xgboost_train_time = xgboost_end_train_time - xgboost_start_train_time

# Predictions
xgboost_start_pred_time = time.time()
xgboost_predicted_valid = xgboost.predict(features_valid)

# Predictions time
xgboost_end_pred_time = time.time()
xgboost_pred_time = xgboost_end_pred_time - xgboost_start_pred_time

In [72]:
# Quality of prediction metrics
xgboost_mse = mean_squared_error(target_valid, xgboost_predicted_valid)
xgboost_rmse = xgboost_mse ** 0.5
xgboost_mape = mean_absolute_percentage_error(target_valid, xgboost_predicted_valid)

# Print XGBoost
print("Modelo XGBoost")
print()

# Print metrics
print("Calidad de predicciones:")
print("Mean Squared Error (MSE):", xgboost_mse)
print("Root Mean Squared Error (RMSE):", xgboost_rmse)
print("Mean Absolute Percentage Error (MAPE):", xgboost_mape)
print()

# Print time
print("Tiempo de entrenamiento:", xgboost_train_time, "seconds")
print("Tiempo de predicción:", xgboost_pred_time, "seconds")

Modelo XGBoost

Calidad de predicciones:
Mean Squared Error (MSE): 1776524.0930035997
Root Mean Squared Error (RMSE): 1332.8631186298162
Mean Absolute Percentage Error (MAPE): 2.163655593663008e+17

Tiempo de entrenamiento: 0.3985927104949951 seconds
Tiempo de predicción: 0.031795501708984375 seconds


-----

## Evaluación del modelo

<div style="color: #196CC4;">
A continuación se muestra un análisis completo de todas las métricas obtenidas en los modelos de entrenamiento con y sin Potenciación de Gradiente<br><br>
<b>Calidad de predicciones</b> <br>   
▶ <b>Mean Squared Error (MSE):</b> es una medida que calcula el promedio de los cuadrados de las diferencias entre los valores predichos por el modelo y los valores reales del objetivo. En este proyecto, el MSE nos indica cuán cerca están las predicciones del precio de los automóviles del valor real. Un MSE más bajo significa que el modelo tiene un mejor ajuste a los datos y predice con mayor precisión los precios de los automóviles.<br>
▶ <b>Root Mean Squared Error (RMSE):</b> es la raíz cuadrada del MSE y proporciona una medida de la magnitud promedio de los errores en la misma unidad que la variable objetivo. Esta métrica nos ayudará evaluar la precisión del modelo en nuestro proyecto. Un RMSE más bajo indica que el modelo tiene una mejor precisión en la predicción de los precios de los automóviles.<br>
▶ <b>Mean Absolute Percentage Error (MAPE):</b> es una métrica de evaluación utilizada en problemas de predicción para medir la precisión de un modelo en términos de porcentaje de error en las predicciones. En este caso cuanto más bajo sea el valor de MAPE, mejor será la capacidad del modelo para predecir con precisión los precios de los automóviles.<br>
▶ <b>Comparativa:</b>El Mean Absolute Percentage Error (MAPE) es una métrica relativa porque calcula el error como un porcentaje del valor real. En contraste con el Mean Squared Error (MSE) o el Root Mean Squared Error (RMSE), que proporcionan una medida absoluta del error en las mismas unidades que la variable de salida, el MAPE mide el error como una proporción del valor real. Esto hace que el MAPE sea una medida relativa que describe la precisión del modelo en relación con el tamaño de los valores reales.<br><br>
<b>Tiempo</b><br>    
▶ <b>Tiempo de entrenamiento:</b> nos indica cuánto tiempo lleva construir y ajustar el modelo a partir de los datos de entrenamiento. Un menor tiempo de entrenamiento es preferible, ya que permite desarrollar y ajustar modelos más rápidamente.<br>
▶ <b>Velocidad de Predicción:</b> nos indica cuánto tiempo lleva al modelo hacer predicciones sobre nuevos datos después de haber sido entrenado. Una velocidad de predicción más rápida es deseable, especialmente en aplicaciones en tiempo real donde las predicciones deben ser generadas rápidamente.<br>
</div>

### Resumen de métricas

<div style="color: #196CC4;">
▶ A continuación se muestran todas las métricas obtenidas para todos los modelos utilizados a lo largo del proyecto
</div>

In [73]:

# Definir los datos de los modelos
modelos = [
    "DecisionTree",
    "Regresión lineal",
    "Random Forest",
    "LGBM",
    "CatBoost",
    "XGBoost"
]

mse = [
    tree_mse,
    rlineal_mse,
    forest_mse,
    lgbm_mse,
    catboost_mse,
    xgboost_mse
]

rmse = [
    tree_rmse,
    rlineal_rmse,
    forest_rmse,
    lgbm_rmse,
    catboost_rmse,
    xgboost_rmse
]

mape = [
    tree_mape,
    rlineal_mape,
    forest_mape,
    lgbm_mape,
    catboost_mape,
    xgboost_mape
]

train_time = [
    tree_train_time,
    rlineal_train_time,
    forest_train_time,
    lgbm_train_time,
    catboost_train_time,
    xgboost_train_time
]

pred_time = [
    tree_pred_time,
    rlineal_pred_time,
    forest_pred_time,
    lgbm_pred_time,
    catboost_pred_time,
    xgboost_pred_time
]

# Crear el DataFrame
df_metrics = pd.DataFrame({
    "Modelo": modelos,
    "Mean Squared Error (MSE)": mse,
    "Root Mean Squared Error (RMSE)": rmse,
    "Mean Absolute Percentage Error (MAPE)": mape,
    "Tiempo de entrenamiento (segundos)": train_time,
    "Tiempo de predicción (segundos)": pred_time
})

display(df_metrics)

,Modelo,Mean Squared Error (MSE),Root Mean Squared Error (RMSE),Mean Absolute Percentage Error (MAPE),Tiempo de entrenamiento (segundos),Tiempo de predicción (segundos)
0,DecisionTree,2.975726e+06,1725.029304,2.325412e+17,0.195555,0.009881
1,Regresión lineal,3.425451e+06,1850.797350,2.388739e+17,0.259417,0.011918
2,Random Forest,2.414589e+06,1553.894761,2.260627e+17,134.914075,0.346035
3,LGBM,1.651853e+06,1285.244315,2.123347e+17,0.446797,0.047137
4,CatBoost,2.001682e+06,1414.808260,2.221495e+17,0.747348,0.000000
5,XGBoost,1.776524e+06,1332.863119,2.163656e+17,0.398593,0.031796


### Análisis de calidad de predicciones

<div style="color: #196CC4;">
<b>MSE (Mean Squared Error):</b><br>
▶ Mide la media de los errores al cuadrado, por lo que valores más bajos indican mejor rendimiento.<br>    
▶ El modelo con el MSE más bajo es LGBM, seguido por XGBoost y Random Forest.<br>
▶ Los modelos DecisionTree y Regresión Lineal muestran un rendimiento aceptable en términos de MSE, pero son superados por otros modelos de conjunto.<br>
    
<b>RMSE (Root Mean Squared Error):</b> <br>
▶ El RMSE es una medida de la dispersión de los errores en las predicciones.<br>
▶ Al igual que con el MSE, el modelo LGBM tiene el RMSE más bajo, seguido por XGBoost y Random Forest.<br>
▶ DecisionTree y Regresión Lineal tienen un RMSE más alto en comparación con los modelos de conjunto.<br> 
    
<b>LGBM muestra el mejor rendimiento en términos de MSE y RMSE, con tiempos de entrenamiento y predicción moderados.
XGBoost y Random Forest también muestran un buen rendimiento en términos de MSE y RMSE, aunque con tiempos de entrenamiento más largos.</b>    
</div>    

### Análisis de velocidad y tiempo

<div style="color: #196CC4;">
<b>Tiempo de Entrenamiento:</b><br>
▶ El modelo con el tiempo de entrenamiento más corto es la Regresión Lineal, seguido por DecisionTree y XGBoost.<br>
▶ LGBM, CatBoost y Random Forest muestran tiempos de entrenamiento más largos en comparación con otros modelos.<br>
▶ A pesar de tener tiempos de entrenamiento más largos, los modelos de conjunto como LGBM, CatBoost y Random Forest proporcionan un mejor rendimiento en términos de calidad de predicción en comparación con modelos más simples como la Regresión Lineal.<br>
    
<b>Velocidad de Predicción:</b><br>
▶ DecisionTree y Regresión Lineal tienen los tiempos de predicción más bajos, seguidos por XGBoost.<br>
▶ LGBM, CatBoost y Random Forest muestran tiempos de predicción más altos en comparación con modelos más simples como DecisionTree y Regresión Lineal.<br>
▶ Aunque DecisionTree y Regresión Lineal tienen tiempos de predicción más bajos, su rendimiento en términos de calidad de predicción es inferior en comparación con los modelos de conjunto más complejos. 
</div>      

## Conclusiones

### Selección del modelo final

<div style="color: #196CC4;">
LGBM ha mostrado consistentemente un buen rendimiento en términos de MSE, RMSE y MAPE en comparación con otros modelos en el conjunto de datos dado. Con un MSE y RMSE más bajos, indica que las predicciones realizadas por LGBM tienden a estar más cerca de los valores reales en promedio. Además, tiene uno de los valores de MAPE más bajos, lo que sugiere una menor discrepancia entre las predicciones y los valores reales en términos de porcentaje:<br>
    
▶ Aunque LGBM tiene tiempos de entrenamiento más largos en comparación con modelos más simples como la Regresión Lineal, su rendimiento en términos de calidad de predicción justifica este costo adicional en tiempo. <br>
▶ Si bien LGBM no es el modelo más rápido en términos de velocidad de predicción, aún tiene tiempos de predicción razonablemente buenos
</div>    

In [77]:
# Hyperparameters
lgbm_final = LGBMRegressor(
    objective='regression',
    num_leaves=35,
    seed=23
)

# Training
lgbm_final_start_train_time = time.time()
lgbm_final.fit(total_features_train, total_target_train)

# Training time
lgbm_final_end_train_time = time.time()
lgbm_final_train_time = lgbm_final_end_train_time - lgbm_final_start_train_time

# Predictions
lgbm_final_start_pred_time = time.time()
lgbm_final_predicted_test = lgbm_final.predict(features_test)

# Predictions time
lgbm_final_end_pred_time = time.time()
lgbm_final_pred_time = lgbm_final_end_pred_time - lgbm_final_start_pred_time

# Quality of prediction metrics
lgbm_final_mse = mean_squared_error(target_test, lgbm_final_predicted_test)
lgbm_final_rmse = lgbm_final_mse ** 0.5
lgbm_final_mape = mean_absolute_percentage_error(target_test, lgbm_final_predicted_test)

# Print LGBM Final Model
print("\nModelo LGBM Final")


# Print metrics
print("Calidad de predicciones:")
print("Mean Squared Error (MSE):", lgbm_final_mse)
print("Root Mean Squared Error (RMSE):", lgbm_final_rmse)
print("Mean Absolute Percentage Error (MAPE):", lgbm_final_mape)
print()

# Print time
print("Tiempo de entrenamiento:", lgbm_final_train_time, "seconds")
print("Tiempo de predicción:", lgbm_final_pred_time, "seconds")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006794 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 346
[LightGBM] [Info] Number of data points in the train set: 216988, number of used features: 24
[LightGBM] [Info] Start training from score 3589.991562

Modelo LGBM Final
Calidad de predicciones:
Mean Squared Error (MSE): 1664534.8743611246
Root Mean Squared Error (RMSE): 1290.1685449433048
Mean Absolute Percentage Error (MAPE): 2.118926661069755e+17

Tiempo de entrenamiento: 0.6681802272796631 seconds
Tiempo de predicción: 0.037001609802246094 seconds


### Conclusiones

<div style="color: #196CC4;">
El modelo LightGBM es la mejor opción para Rusty Bargain en términos de calidad de predicción, velocidad de predicción y eficiencia en el tiempo de entrenamiento. Aunque el modelo tiene un tiempo de entrenamiento ligeramente mayor que otros modelos más simples, como la regresión lineal, su capacidad para generar predicciones precisas y su tiempo de respuesta rápido lo hacen ideal para su implementación en la plataforma.<br>
    
Este modelo cumple con los criterios establecidos por Rusty Bargain, proporcionando una herramienta valiosa para que los usuarios obtengan estimaciones confiables y rápidas del valor de mercado de sus vehículos. La implementación de este modelo en la aplicación mejorará significativamente la experiencia del usuario, atrayendo nuevos clientes y potenciando la propuesta de valor de Rusty Bargain en el mercado de autos usados.
</div>    